In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import os
import sqlite3
import asyncio
import re

from playwright.async_api import Playwright, async_playwright, expect


# # Append the parent directory to sys.path
# parent_dir = os.path.abspath('..')
# sys.path.append(parent_dir)

# from src.main import main

# Validierung
Validation start with the generated playwrite code of the LLM.
The generated script will be executed and processed in order to determine the evaluation.

In [27]:
print(f"Current WD: {os.getcwd()}")
TEST_SCRIPT = "./test_script/playwright_testing.py"
SCREENSHOT = "./screenshot/"
file_name = TEST_SCRIPT.split("/")[-1].split(".")[0] + ".png"
path_screenshot = SCREENSHOT + file_name
path_html = "./HTML/test.html"
print(path_screenshot)

Current WD: /Users/frbroy/Library/Mobile Documents/com~apple~CloudDocs/KIT/SoSe2024/PSDA/cadenza-playwright-llm/notebooks
./screenshot/playwright_testing.png


In [28]:
sc_path = "safe.png"
# command to add screenshot
screenshot_code = f"    await page.screenshot(path = \"{sc_path}\")\n"
# Construct the HTML extraction command with proper indentation
html_extraction_command = '    page_content = await page.content()\n'
html_extraction_command += '    with open("{}", "w") as file:\n'.format(path_html)
html_extraction_command += '        file.write(page_content)\n\n'
# Ensure the script exists
if not os.path.exists(TEST_SCRIPT):
    print(f"File not found: {TEST_SCRIPT}")
else:
    # Read and execute the Playwright script
    with open(TEST_SCRIPT, 'r') as file:
        script_content = file.readlines()
        #exec(script_content)

    # Find the position to insert the screenshot command
    insert_position = 0
    for i, line in enumerate(script_content):
        if "await page.close()" in line or "await context.close()" in line or "await browser.close()"in line:
            insert_position = i 
            break

    # Insert the screenshot command
    script_content.insert(insert_position, screenshot_code)

    # Join the script content back into a single string
    modified_script_content = ''.join(script_content)
    exec(modified_script_content)

RuntimeError: asyncio.run() cannot be called from a running event loop

In [24]:
print(modified_script_content)

import asyncio
import re
from playwright.async_api import Playwright, async_playwright, expect


async def run(playwright: Playwright) -> None:
    browser = await playwright.chromium.launch(headless=False)
    context = await browser.new_context(storage_state="./context/context__e210490e-87e5-462b-86b7-3879adc487be.json")
    page = await context.new_page()
    await page.goto("http://localhost:8080/cadenza/login")
    await page.get_by_label("Benutzername *").click()
    await page.get_by_label("Benutzername *").fill("Admin")
    await page.get_by_label("Benutzername *").press("Tab")
    await page.get_by_placeholder(" ").fill("Admin")
    await page.get_by_placeholder(" ").press("Enter")
    await page.get_by_role("link", name="Management Center").click()
    await page.get_by_role("link", name="Arbeitsmappen").click()
    await page.get_by_role("link", name="Arbeitsmappe Zugangsdaten").click()
    await page.locator("d-worksheet-view").filter(has_text="Nutzer: CreatorPasswort:").lo

In [11]:
sc_path = "safe.png"
screenshot = f"await page.screenshot(path = \"{path_screenshot}\")"
screenshot

'await page.screenshot(path = "safe.png")'

# Load Data

In [10]:
# Connect to the database
conn = sqlite3.connect('../data/raw/playwright_script.db')
cursor = conn.cursor()

res = cursor.execute("SELECT * FROM tests")
items = res.fetchall()

print("There are {} data.".format(len(items)))

# Query the sqlite_master table to get the table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
table = tables[0]

table_name = table[0]
print(f"\nStructure of table '{table_name}':")
cursor.execute(f"PRAGMA table_info({table_name});")
columns = cursor.fetchall()
for column in columns:
    print(column)

There are 100 data.

Structure of table 'tests':
(0, 'id', 'TEXT', 0, None, 0)
(1, 'steps', 'TEXT', 0, None, 0)
(2, 'expectation', 'TEXT', 0, None, 0)
(3, 'html', 'TEXT', 0, None, 0)
(4, 'screenshot', 'TEXT', 0, None, 0)
(5, 'test_script', 'TEXT', 0, None, 0)


In [11]:
# Check the first item
items[0]

('1.1',
 '[1.1] Öffne die Arbeitsmappe "Übersicht Messstellen" im Ordner "Gewässergüte".',
 '[1.1] Expected result: Die Arbeitsmappe wird geöffnet, der Analysekontext ist nicht sichtbar.',
 '.\\html\\1_1.html',
 '.\\screenshot\\1_1.png',
 '.\\test_script\\1_1.spec.ts')

# Run UI Test Generation locally (smaller model GPT-2)

In [12]:
def get_previous_id(id):
    test, step = map(int, id.split('.'))
    if step > 1:
        previous_id = f"{test}.{step - 1}"
    else:
        print(f"Test {test} has no more previous step. No context available.")

    return previous_id

In [13]:
def fetch_relevant_items(db_file, current_id):
    # Connect to SQLite database
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Get the previous ID
    previous_id = get_previous_id(current_id)

    # Prepare the SQL query to retrieve the desired rows
    query = 'SELECT * FROM tests WHERE id IN (?, ?)'
    cursor.execute(query, (current_id, previous_id))

    # Fetch all matching rows
    items = cursor.fetchall()

    # Close the connection
    conn.close()

    return items

In [14]:
def map_items_to_args(items):
    # The file paths from the test x-1 are used as context
    html_path = items[0][3]
    image_path = items[0][4]
    precondition_path = items[0][5]
    # The last step of the text x is used as prompt
    steps = items[1][1].split(']')
    prompt = steps[-1].strip()

    return {"html_path": html_path,
            "image_path": image_path,
            "precondition_path": precondition_path,
            "prompt": prompt}

Now we can run the main function with the extracted information, we will get the generated UI test code for the test with id 1.4.

In [15]:
# Select test ID and database file
db_file = '../data/raw/playwright_script.db'
current_id = '1.4'

In [16]:
items = fetch_relevant_items(db_file, current_id)
args = map_items_to_args(items)

In [17]:
items

[('1.3',
  '[1.1] Öffne die Arbeitsmappe "Übersicht Messstellen" im Ordner "Gewässergüte". [1.2]  Öffnen der Tabellen-Sicht "Messstellenliste" über die Werkzeugliste der Arbeitsmappe. [1.3] Klicke auf den 3-Punkte-Button innerhalb der Tabellen-Sicht "Messstelleninformationen"',
  '[1.3] Expected result: Kontextmenü erscheint mit den Optionen: - Duplizieren - In Arbeitsblatt duplizieren - Exportieren - Designer öffnen - Löschen.',
  '.\\html\\1_3.html',
  '.\\screenshot\\1_3.png',
  '.\\test_script\\1_3.spec.ts'),
 ('1.4',
  '[1.1] Öffne die Arbeitsmappe "Übersicht Messstellen" im Ordner "Gewässergüte". [1.2]  Öffnen der Tabellen-Sicht "Messstellenliste" über die Werkzeugliste der Arbeitsmappe. [1.3] Klicke auf den 3-Punkte-Button innerhalb der Tabellen-Sicht "Messstelleninformationen". [1.4] Klicke auf "Duplizieren".',
  '[1.4] Expected result: Das Kontextmenü schließt sich. Die Sicht ist dupliziert. Die duplizierte Sicht erscheint neben der Original-Sicht. Beide Sichten teilen sich in

In [23]:
import os
os.getcwd()

'/Users/frbroy/Library/Mobile Documents/com~apple~CloudDocs/KIT/SoSe2024/PSDA/cadenza-playwright-llm/notebooks'

In [24]:
%%time
# Append the parent directory to sys.path
parent_dir = os.path.abspath('./')
print(parent_dir)
sys.path.append(parent_dir)


main(**args)

/Users/frbroy/Library/Mobile Documents/com~apple~CloudDocs/KIT/SoSe2024/PSDA/cadenza-playwright-llm/notebooks
2024-06-20 10:57:39 [src.main:24] [INFO] >>>> Loading context...


FileNotFoundError: [Errno 2] No such file or directory: '.\\html\\1_3.html'

# More Developing: Using modular source code
Possible to update function content in the following files, but no RENAMING or DELETING or changing function signature!:
* src/main.py: Main function for UI test generation.
    * Change constant Max length (max length of the input text for the model)
* src/ui_tests/test_generation.py: Generate the UI test code using the language model.
    * Expand model selection: Just add new case statements for different models with its code.
* src/data/html_processing.py: Extract relevant information from the HTML file.
* src/data/image_processing.py: Extract relevant information from the image.
* src/data/python_processing.py: Parse the given playwright test code for previous step as a precondition.
